# Mount Google Drive and Navigate to Code

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/My Drive/bio_research/Code')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Install Colab Dependencies

In [2]:
!pip install segmentation-models
!pip install albumentations==0.4.5
!pip install keras===2.3.1
!pip install mahotas
!pip install segutils
!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 122kB 6.8MB/s 
     |████████████████████████████████| 634kB 7.7MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=07109a62148d5435ea585b395ec63996822a85951e9488dda10ac656a71949e7
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=5ae752fa7737d735c48111f11712958228dfafa53b56da9aaca650b3011550e0
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.

# Imports

In [3]:
# io imports
import os
import tifffile
import json

# preprocessing imports
from MicroscopyPreprocessor import MicroscopyPreprocessor

# segmentations imports
import keras_seg as seg

# keras imports
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam

# jupyter auto reload
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

Segmentation Models: using `keras` framework.


# Tunable Parameters

In [4]:
# model
BACKBONE = seg.models.Backbone.RESNET50
PRETRAINED_WEIGHTS = True
MODEL = seg.models.FPN

# training
LOSS = seg.losses.bce_jaccard_loss
INITIAL_LEARNING_RATE = 0.001
LEARNING_RATE_DECAY_FACTOR = 0.1
LEARNING_RATE_DECAY_STEP = 10
EPOCHS = 30

DROPOUT = 0.5

In [5]:
# create place to save output for this run
save_dir = os.path.join(
    '..',
    'Docs',
    'segmentation fluo results',
    'train_test_res',
    '{}_{}_use-imagenet.{}_epochs.{}_loss.{}_lr.{}_decay.{}_step.{}_dropout.{}---->>>>'.format(MODEL.__name__,
                                                                                       BACKBONE,
                                                                                       PRETRAINED_WEIGHTS,
                                                                                       EPOCHS,
                                                                                       LOSS.name,
                                                                                       INITIAL_LEARNING_RATE,
                                                                                       LEARNING_RATE_DECAY_FACTOR,
                                                                                       LEARNING_RATE_DECAY_STEP,
                                                                                       DROPOUT)
)
os.makedirs(save_dir, exist_ok=True)

# Read Data

In [6]:
images_dir = os.path.join('resources', 'images', 'fluo_seg', 'all')

prep = MicroscopyPreprocessor(BACKBONE)

train_seq = seg.datasets.SimpleDataset(os.path.join(images_dir, 'train'), transforms=prep.training)
val_seq = seg.datasets.SimpleDataset(os.path.join(images_dir, 'test'), transforms=prep.validation)

# Initialize Model

In [7]:
# initialize  new model for fold
if MODEL == seg.models.FPN:
    model = MODEL(backbone=BACKBONE, num_channels=3, use_pretrained_imagenet_weights=PRETRAINED_WEIGHTS, dropout=DROPOUT)
else:
    model = MODEL(backbone=BACKBONE, num_channels=3, use_pretrained_imagenet_weights=PRETRAINED_WEIGHTS)

model.compile(loss=LOSS,
              optimizer=Adam(),
              metrics=[
                  'accuracy',
                  seg.metrics.iou_score,
                  seg.metrics.precision,
                  seg.metrics.recall,
                  seg.metrics.f1_score,
                  seg.metrics.f2_score
              ])

Instructions for updating:
Colocations handled automatically by placer.
94593024/94592056 [==============================] - 4s 0us/step


# Train

In [8]:
title = f'Training model'
print(f'\n\n{"=" * len(title)}\n{title}\n{"=" * len(title)}\n\n')

lr_scheduler = LearningRateScheduler(seg.lr_schedulers.StepScheduler(
    INITIAL_LEARNING_RATE,
    LEARNING_RATE_DECAY_FACTOR,
    LEARNING_RATE_DECAY_STEP
))

# create checkpoint for saving weights
checkpoint = ModelCheckpoint(filepath=os.path.join(save_dir, f'model_weigths.hd5'),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')

import time

start = time.time()

history = model.fit_generator(
    generator=train_seq,
    epochs=EPOCHS,
    callbacks=[checkpoint, lr_scheduler],
    validation_data=val_seq,
)

end = time.time()

print('total training time: {}'.format(end - start))



Training model


Instructions for updating:
Use tf.cast instead.
Epoch 1/30
27/27 [==============================] - 75s 3s/step - loss: 1.1662 - accuracy: 0.9015 - iou_score: 0.0197 - precision: 0.0223 - recall: 0.2936 - f1-score: 0.0370 - f2-score: 0.0655 - val_loss: 1.1965 - val_accuracy: 0.9991 - val_iou_score: 0.0010 - val_precision: 0.0010 - val_recall: 0.1476 - val_f1-score: 0.0021 - val_f2-score: 0.0050

Epoch 00001: val_loss improved from inf to 1.19645, saving model to ../Docs/segmentation fluo results/train_test_res/FPN_Backbone.RESNET50_use-imagenet.True_epochs.30_loss.binary_crossentropy_plus_jaccard_loss_lr.0.001_decay.0.1_step.10_dropout.0.5---->>>>/model_weigths.hd5
Epoch 2/30
27/27 [==============================] - 52s 2s/step - loss: 0.8116 - accuracy: 0.9982 - iou_score: 0.1960 - precision: 0.2737 - recall: 0.5766 - f1-score: 0.3151 - f2-score: 0.4064 - val_loss: 1.0064 - val_accuracy: 0.9991 - val_iou_score: 0.0013 - val_precision: 0.0277 - val_recall: 0.0015 - v

# Save and Plot

In [ ]:
# save history as text
with open(os.path.join(save_dir, f'history.json'), 'w') as f:
    json.dump(eval(str(history.history)), f, indent=4)

# plot all metrics
save_path_template = os.path.join(save_dir, '{} plot.png')

seg.vis.plot_history(history, 'loss', save_path_template.format(LOSS.name + ' loss'))
seg.vis.plot_history(history, 'accuracy', save_path_template.format('accuracy'))
seg.vis.plot_history(history, seg.metrics.iou_score.name, save_path_template.format(seg.metrics.iou_score.name))
seg.vis.plot_history(history, seg.metrics.precision.name, save_path_template.format(seg.metrics.precision.name))
seg.vis.plot_history(history, seg.metrics.recall.name, save_path_template.format(seg.metrics.recall.name))
seg.vis.plot_history(history, seg.metrics.f1_score.name, save_path_template.format(seg.metrics.f1_score.name))
seg.vis.plot_history(history, seg.metrics.f2_score.name, save_path_template.format(seg.metrics.f2_score.name))